### Como manipular XML com XPath

In [2]:
from gensim.models import KeyedVectors
import unicodedata

import gzip
import xml.etree.ElementTree as ET

with gzip.open('pordesc2018-small.xml.gz') as pordesc2018:
    tree = ET.parse(pordesc2018)

### Monta Dicionario

In [3]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [5]:
dictMesh = {}
for d in tree.findall("./DescriptorRecord"):
    terms = []
    
    for aql in d.findall('.AllowableQualifiersList/AllowableQualifier/QualifierReferredTo/QualifierName'):
        teste_qualifier = aql.find('./String').text
        if(teste_qualifier == 'anatomy & histology' or teste_qualifier == 'pharmacology' or teste_qualifier == 'methods' or teste_qualifier == 'diagnosis'):
            qualifier = teste_qualifier
            break
        else:
            qualifier = "#"
    
    for c in d.findall('.ConceptList/'):
        if c.find('./ScopeNote') != None:
            scope = c.find('./ScopeNote').text.replace('\n','').strip()
        for t in c.findall('./TermList/'):
            terms.append(t.find('./String').text)

    ## Busca palavras similares oas termos no Word Embedding e adiciona nos termos:
    termos_similares = []
    
    #Usa isso somente porque o dicionario tem os nomes em portugues
    
    name = d.find('.DescriptorName/String').text
    name = name.split('[')
    name_new = name[1].replace(']', '')
    
            
    dictMesh[d.find('.DescriptorUI').text] = {
        'ID': d.find('.DescriptorUI').text,
        'name': name_new,
        'scope': scope,
        'terms': set(terms),
        'qualifier': qualifier
    }
            
    #break
len(dictMesh)

1859

### Salva Dicionario

In [16]:
import gzip, pickle

with gzip.open('dictMesh.dict_eng.gz','wb') as fp:
    pickle.dump(dictMesh,fp)
    fp.close()

### Carrega Dicionario

In [17]:
import gzip, pickle

with gzip.open('dictMesh.dict_eng.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()
    
dictMesh['D000001']

{'ID': 'D000001',
 'name': 'Calcimycin',
 'scope': 'An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.',
 'terms': {'A 23187',
  'A-23187',
  'A23187',
  'A23187, Antibiotic',
  'Antibiotic A23187',
  'Calcimicina',
  'Calcimycin'},
 'qualifier': 'pharmacology'}

In [6]:
palavra = 'Calcimicina'
for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if t.lower() == palavra.lower():
            print(t)
            print(d['scope'])

Calcimicina
An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.
